In [2]:
import cirq
import numpy as np

# The Portfolio Rebalancing Optimization using QAOA with soft constraints

In this notebook, we will apply the QAOA algorithm to solve the portfolio optimization problem using a soft constraint


$$
C_{RR}({\bf z}) = \sum_i \lambda r_i z_i - (1-\lambda)\sum_{ij} \sigma_{ij} z_i z_j
$$

$$
C_{TC}({\bf z}) = \sum_i T (1- \delta(z-y))
$$


The soft constraint that we impose will be given 

$$
H =C_{RR}({\bf z})+C_{TC}({\bf z})+A\left(D-\sum_i z_i \right)^2
$$

In [4]:
# Initialize the states 

# define the length of the grid.
length = 3
# define qubits on the grid.
qubits = [cirq.GridQubit(i, j) for i in range(length) for j in range(length)]

print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]
